In [110]:
%pip install --upgrade e2b_code_interpreter cohere==5.5.3

Note: you may need to restart the kernel to use updated packages.


In [111]:
# Instantiate the Cohere client

import cohere
import os
import sys
import json

import os
from dotenv import load_dotenv
load_dotenv()

COHERE_API_KEY = os.environ["COHERE_API_KEY"]
E2B_API_KEY = os.getenv("E2B_API_KEY")

co = cohere.Client(api_key=COHERE_API_KEY)

In [112]:
SYSTEM_PROMPT = """
## your job & context
you are a professional personal assistant. you are given tasks to complete and you run python code to solve them.
You DO NOT MAKE SYNTAX MISTAKES OR FORGET ANY IMPORTS
- the python code runs in jupyter notebook.
- every time you call `execute_python` tool, the python code is executed in a separate cell. it's okay to multiple calls to `execute_python`.
- display visualizations using matplotlib or any other visualization library directly in the notebook. don't worry about saving the visualizations to a file.
- you have access to the internet and can make api requests.
- you also have access to the filesystem and can read/write files.
- you have access to various tools.
- you can install any pip package (if it exists) if you need to but the usual packages for data analysis are already preinstalled.
- you can run any python code you want, everything is running in a secure sandbox environment.
"""

USER_PROMPT = "Book an hour long appointment for the first available free slot after 3pm. Using puthon code, generate a chart with timeline showing the events in calendar."

In [113]:
# Define the tools

def list_calendar_events(date: str):
  events = '[{"start": "14:00", "end": "15:00"}, {"start": "15:00", "end": "16:00"}, {"start": "17:00", "end": "18:00"}]'
  print(f"Listing events: {events}")
  return events

def create_calendar_event(date: str, time: str, duration: int):
  print(f"Creating a {duration} hour long event at {time} on {date}")
  return True


def code_interpret(e2b_code_interpreter, code):
  print("Running code interpreter...")
  exec = e2b_code_interpreter.notebook.exec_cell(
    code,
    # Stream stdout and stderr from the Code Interpreter
    on_stdout=lambda stdout: print("[Code Interpreter]", stdout),
    on_stderr=lambda stderr: print("[Code Interpreter]", stderr),
    # You can also stream code execution results
    # on_result=...
  )

  if exec.error:
    print("[Code Interpreter ERROR]", exec.error)
  else:
    return exec

list_calendar_events_tool = {
  "name": "list_calendar_events",
  "description": "returns a list of calendar events for the specified date, including the start time and end time for each event",
  "parameter_definitions": {
    "date": {
      "description": "the date to list events for, formatted as mm/dd/yy",
      "type": "str",
      "required": True
    }
  }
}

create_calendar_event_tool = {
  "name": "create_calendar_event_tool",
  "description": "creates a calendar event of the specified duration at the specified time and date",
  "parameter_definitions": {
    "date": {
      "description": "the date on which the event starts, formatted as mm/dd/yy",
      "type": "str",
      "required": True
    },
    "time": {
      "description": "the time of the event, formatted using 24h military time formatting",
      "type": "str",
      "required": True
    },
    "duration": {
      "description": "the number of hours the event lasts for",
      "type": "float",
      "required": True
    }
  }
}

execute_python_tool =   {
  "name": "execute_python",
  "description": "Execute python code in a Jupyter notebook cell and returns any result, stdout, stderr, display_data, and error.",
  "parameter_definitions": {
      "code": {
          "description": "The python code to execute in a single cell.",
          "type": "str",
          "required": True
    }
  },
}


# helper function for routing to the correct tool
def invoke_tool(tool_call: cohere.ToolCall):
  if tool_call.name == list_calendar_events_tool["name"]:
    date = tool_call.parameters["date"]
    return [{
        "events": list_calendar_events(date)
    }]
  elif tool_call.name == create_calendar_event_tool["name"]:
    date = tool_call.parameters["date"]
    time = tool_call.parameters["time"]
    duration = tool_call.parameters["duration"]

    return [{
        "is_success": create_calendar_event(date, time, duration)
    }]
  elif tool_call.name == execute_python_tool["name"]:
    code = tool_call.parameters["code"]

    return [{
        "is_success": code_interpret(e2b_code_interpreter, code)  # TBD Fix
    }]
  else:
    raise f"Unknown tool name '{tool_call.name}'"

In [114]:
# Check what tools the model wants to use and how to use them
# Define chat with model

def chat(e2b_code_interpreter, user_message):
  print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

  res = co.chat(
      model="command-r-plus",
      preamble="Today is Thursday, may 23, 2024",
      message=USER_PROMPT,
      force_single_step=False,
      tools=[list_calendar_events_tool, create_calendar_event_tool, execute_python_tool])

  while res.tool_calls:
    print(res.text) # This will be an observation and a plan with next steps

    # invoke the recommended tools
    tool_results = []
    for call in res.tool_calls:
      # Add print statements to inspect the variables
      print()
      print(f"🚨 MONITORING call: {call}")
      print()
      print(f"🚨 invoke_tool(call): {invoke_tool(call)}")
      print()

      tool_results.append({"call": call, "outputs": invoke_tool(call)})

    # send back the tool results
    res = co.chat(
      model="command-r-plus",
      chat_history=res.chat_history,
      message="",
      force_single_step=False,
      tools=[list_calendar_events_tool, create_calendar_event_tool, execute_python_tool],
      tool_results=tool_results,
    )
  for choice in res.chat_history:   # TBD: Maybe it's not chat history? What is the correct attribute? (Instead of choices)
    if choice.message.tool_calls and len(choice.message.tool_calls) > 0:
      for tool_call in choice.message.tool_calls:
        if tool_call.function.name == "execute_python":
          if "code" in tool_call.function.arguments:
            args = json.loads(tool_call.function.arguments)
            code = args["code"]          
            print("CODE TO RUN")
            print(code)
            execution_output = code_interpret(e2b_code_interpreter, code)
            return execution_output
    else:
      print("Answer:", choice.message.content)


#  print(res.text) # print the final answer
  

In [116]:
# Print the results

from e2b_code_interpreter import CodeInterpreter

with CodeInterpreter(api_key=E2B_API_KEY) as code_interpreter:
  execution_output = chat(
    code_interpreter,
    USER_PROMPT,
  )

  if execution_output == None:
    print("No output from the code interpreter. Did the LLM generate any code?")
    sys.exit(1)

  # Access stdout
  print("Stdout:", execution_output.logs.stdout)
  # Access stderr
  print("Stderr:", execution_output.logs.stderr)

  # Access any runtime errors
  print("AI-generated Python runtime error:", execution_output.error)

  # Access any results of code execution - charts, interpreter last line, images, etc.
  print("Results:", execution_output.results)
  if len(execution_output.results) == 0:
    sys.exit(0)

execution_output.results[0]


User Message: Book an hour long appointment for the first available free slot after 3pm. Using puthon code, generate a chart with timeline showing the events in calendar.
First, I will use the list_calendar_events tool to find the first available free slot after 3pm. Then, I will use the create_calendar_event_tool to book an hour-long appointment at that time. Finally, I will use Python code to generate a chart with a timeline showing the events in the calendar.

🚨 MONITORING call: name='list_calendar_events' parameters={'date': '05/23/2024'}

Listing events: [{"start": "14:00", "end": "15:00"}, {"start": "15:00", "end": "16:00"}, {"start": "17:00", "end": "18:00"}]
🚨 invoke_tool(call): [{'events': '[{"start": "14:00", "end": "15:00"}, {"start": "15:00", "end": "16:00"}, {"start": "17:00", "end": "18:00"}]'}]

Listing events: [{"start": "14:00", "end": "15:00"}, {"start": "15:00", "end": "16:00"}, {"start": "17:00", "end": "18:00"}]
The first available free slot after 3pm is 4pm. Now,

NameError: name 'e2b_code_interpreter' is not defined